In [6]:
%matplotlib widget

import os
import sys

sys.path.insert(0, os.getenv("HOME") + "/pycode/MscThesis/")
import pandas as pd
from amftrack.util.sys import (
    get_dates_datetime,
    get_dirname,
    temp_path,
    get_plate_number,
    get_postion_number,
)

import ast
from amftrack.plotutil import plot_t_tp1
from scipy import sparse
from datetime import datetime
from amftrack.pipeline.functions.node_id import orient
import pickle
import scipy.io as sio
from pymatreader import read_mat
from matplotlib import colors
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi
from skimage import filters
from random import choice
import scipy.sparse
import os
from amftrack.pipeline.functions.extract_graph import (
    from_sparse_to_graph,
    generate_nx_graph,
    sparse_to_doc,
)
from skimage.feature import hessian_matrix_det
from amftrack.pipeline.functions.experiment_class_surf import Experiment
from amftrack.pipeline.paths.directory import (
    run_parallel,
    find_state,
    directory_scratch,
    directory_project,
    path_code,
)

from amftrack.notebooks.analysis.data_info import *
import matplotlib.patches as mpatches
from statsmodels.stats import weightstats as stests

In [7]:
window = 800
results = {}
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        results[inst] = pickle.load(
            open(f"{path_code}/MscThesis/Results/straight_{window}_{inst}.pick", "rb")
        )

FileNotFoundError: [Errno 2] No such file or directory: '/home/cbisot/pycode//MscThesis/Results/straight_800_(10, 0, 10).pick'

In [8]:
column_names = [
    "plate",
    "inst",
    "treatment",
    "angle",
    "curvature",
    "density",
    "growth",
    "speed",
    "straightness",
    "t",
    "hyph",
]
infos = pd.DataFrame(columns=column_names)
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        (
            angles,
            curvatures,
            densities,
            growths,
            speeds,
            tortuosities,
            ts,
            hyphs,
        ) = results[inst]
        for i, angle in enumerate(angles):
            new_line = pd.DataFrame(
                {
                    "plate": [plate_number[inst]],
                    "inst": [inst],
                    "treatment": [treatment],
                    "angle": [angle],
                    "curvature": [curvatures[i]],
                    "density": [densities[i]],
                    "growth": [growths[i]],
                    "speed": [speeds[i]],
                    "straightness": [tortuosities[i]],
                    "t": [ts[i]],
                    "hyph": [hyphs[i]],
                }
            )  # index 0 for
            # mothers need to be modified to resolve multi mother issue
            infos = infos.append(new_line, ignore_index=True)

KeyError: (10, 0, 10)

In [9]:
corrected = infos.loc[infos["straightness"] <= 1]

In [10]:
corrected

,plate,inst,treatment,angle,curvature,density,growth,speed,straightness,t,hyph
0,296,"(9, 0, 11)",25,6.864531,0.011940,1.973913,574.929348,143.732337,0.997008,5.316667,65
1,296,"(9, 0, 11)",25,3.628532,0.006586,1.834783,550.984831,137.746208,0.988864,9.316667,65
2,296,"(9, 0, 11)",25,-2.438101,-0.003874,2.368116,629.319466,157.329867,0.992084,13.316667,65
3,296,"(9, 0, 11)",25,3.433630,0.004524,2.665217,759.005213,189.751303,0.997847,17.316667,65
4,296,"(9, 0, 11)",25,-18.883147,-0.025417,3.871014,742.928653,185.732163,0.990694,21.316667,65
...,...,...,...,...,...,...,...,...,...,...,...
2211,351,"(35, 0, 15)",25,12.425943,0.020830,1.286957,596.533199,149.133300,0.973802,56.016667,17351
2212,351,"(35, 0, 15)",25,-6.674771,-0.012112,4.501449,551.091249,137.772812,0.974202,56.016667,17566
2213,351,"(35, 0, 15)",25,16.831669,0.033944,2.695652,495.858547,123.450261,0.993517,52.000000,18328
2214,351,"(35, 0, 15)",25,11.477627,0.024167,2.403623,474.929134,118.732283,0.989865,56.016667,18328


In [11]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
bplot1 = corrected.boxplot(
    column=["speed"],
    by="plate",
    figsize=(9, 8),
    ax=ax,
    patch_artist=True,
    showfliers=False,
)
colors = ["lightblue"] + ["pink"] + ["lightgreen"]
for i, (artist, col) in enumerate(zip(ax.artists, colors)):
    artist.set_edgecolor(col)
    artist.set_facecolor(col)
ax.set_xlabel("Plate")
ax.set_ylabel("Speed")
ax.set_ylim(0.9)
plt.show()

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [38]:
max_speeds = []
total_growth = []
for treatment in treatments.keys():
    insts = treatments[treatment]
    for inst in insts:
        inst_tab = corrected.loc[corrected["inst"] == inst]
        for hyph in set(inst_tab["hyph"]):
            max_speeds.append(np.max(inst_tab.loc[inst_tab["hyph"] == hyph]["speed"]))
            total_growth.append(
                np.sum(inst_tab.loc[inst_tab["hyph"] == hyph]["growth"])
            )

In [29]:
len(max_speeds)

679

In [40]:
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(9, 4))
ax.scatter(np.log(total_growth), max_speeds)
# ax.set_xlim(100,300)

/home/cbisot/anaconda3/envs/test/lib/python3.7/site-packages/matplotlib/__init__.py:880: MatplotlibDeprecationWarning: 
nbagg.transparent
  version, key, obj_type="rcparam", alternative=alt_key)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …